### Convert csv to images for data augmentation purposes

In [1]:
import numpy as np
import os
import pandas as pd
from imageio import imwrite

TRAIN_PATH = "data/train"
VALID_PATH = "data/valid"
TEST_PATH = "data/test"
DATASET_PATHS = (TRAIN_PATH, VALID_PATH, TEST_PATH)

IMG_SIZE = 48
OUTPUTS = 7

In [ ]:
df = pd.read_csv("data/fer2013.csv")

In [ ]:
# Create directory for training and validation
for dataset_path in DATASET_PATHS:
    # Create directory if it doesn't exist
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)

    # For each target that exists
    for target in df["emotion"].unique():
        # Make path for each target
        path_target = os.path.join(dataset_path, str(target))
        # Create directroy if it doesn't exist
        if not os.path.exists(path_target):
            os.mkdir(os.path.join(path_target))

# Iterate through all of the data
for i, row in df.iterrows():
    # Convert image to numpy array and reshape to be 2d
    img = np.array(list(map(int, row["pixels"].split(" "))))
    img = img.reshape((IMG_SIZE, IMG_SIZE)).astype(np.uint8)

    # File name is <number>.png
    fname = str(i)+".png"

    # Make path go to validation directory if less than VALID_PROP
    if row["Usage"] == "Training":
        path = os.path.join(TRAIN_PATH, str(row["emotion"]))
    elif row["Usage"] == "PublicTest":
        path = os.path.join(VALID_PATH, str(row["emotion"]))
    elif row["Usage"] == "PrivateTest":
        path = os.path.join(TEST_PATH, str(row["emotion"]))
    else:
        raise Exception("Invalid Usage: {}".format(row["Usage"]))
        
    # Save image to path
    imwrite(os.path.join(path, fname), img)

### Construct CNN architectures

In [15]:
from keras.applications.densenet import dense_block, transition_block
from keras.applications.resnet50 import identity_block, conv_block
from keras.layers import Activation, AveragePooling2D, BatchNormalization, concatenate, Conv2D
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam

class CNN:
    """Abstract base class for all CNNS"""
    
    model = None

    def __init__(self, lr=0.001, decay=0):
        # Use Adam for optimizer with configurable lr and decay
        opt = Adam(lr=lr, decay=decay)
        # Compile model and use crossentropy with accuracy as a metric
        self.model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

    def Conv2D_bn(self, x, nb_filter, filter_size, strides=(1, 1), padding="same"):
        """2D Convolutional Batch Normalization block"""
        # Convolutional layer with configurable filters, strides, and padding
        x = Conv2D(nb_filter, (filter_size, filter_size), strides=strides, padding=padding)(x)
        # Use batch norm to normalize the inputs for the next layer
        x = BatchNormalization()(x)
        # Use ReLU as nonlinearity
        return Activation("relu")(x)


class VGG(CNN):
    """Model based on VGG"""
    
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        # Create vgg_block with 2 convolutions with 64 filters
        x = self.vgg_block(img_input, 2, 64)
        # Create vgg_block with 2 convolutions with 128 filters
        x = self.vgg_block(x, 2, 128)
        # Create vgg_block with 3 convolutions with 256 filters
        x = self.vgg_block(x, 3, 256)
        # Create vgg_block with 3 convolutions with 512 filters
        x = self.vgg_block(x, 3, 512)
        # Flatten convolutional output to fit with following dense layer
        x = Flatten()(x)
        # Create dense batch norm layer with 1024 units
        x = self.Dense_bn(x, 1024)
        # Use dropout with configurable p
        x = Dropout(dropout)(x)
        # Create dense batch norm layer with 1024 units
        x = self.Dense_bn(x, 1024)
        # Use dropout with configurable p
        x = Dropout(dropout)(x)
        # Dense prediction layer with softmax
        predictions = Dense(outputs, activation="softmax")(x)
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

    def vgg_block(self, x, nb_conv, nb_filters, filter_size=3):
        """VGG Block will place nb_conv convolutional layers each with nb_filters filters.
           It is followed by a max pooling layer that halves the height and width.
        """
        # Create nb_conv convolutions
        for i in range(nb_conv):
            x = self.Conv2D_bn(x, nb_filters, filter_size)
        # Use 2x2 max pooling with stride 2x2
        return MaxPooling2D((2, 2), strides=(2, 2))(x)
            
    def Dense_bn(self, x, units):
        """Dense layer followed by Batch Normalization and ReLU"""
        # Create dense layer with configurable units
        x = Dense(units)(x)
        # Use batch norm to normalize the inputs for the next layer
        x = BatchNormalization()(x)
        # Use ReLU as nonlinearity
        return Activation("relu")(x)


class Inception_FCN(CNN):
    """Model that utilizes Google's Inception block"""
    
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        # Create 4 inception blocks to reduce activation map to height and width of 3x3
        x = self.inception_block(img_input)
        x = self.inception_block(x)
        x = self.inception_block(x)
        x = self.inception_block(x)
        # Use dropout to reduce overfitting
        x = Dropout(dropout)(x)
        # Use convolutional layer with filters same as classes to produce outputs
        x = Conv2D(outputs, (3, 3), padding="same")(x)
        # Use GlobalAveragePooling to convert 3x3x3 activations to 1x3 for output
        x = GlobalAveragePooling2D()(x)
        # Use softmax layer
        predictions = Activation("softmax")(x)
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

    def inception_block(self, x):
        """Inception block from GoogleNet paper.
           Uses filters of different sizes and combines their activations,
           which helps it look for various features in an image.
        """
        # Do convolution with 1x1 filter stride 2x2
        branch1x1 = self.Conv2D_bn(x, 64, 1, strides=(2, 2))

        # Do convolution with 1x1 filter
        branch5x5 = self.Conv2D_bn(x, 48, 1)
        # Do convolution with 5x5 filter stride 2x2
        branch5x5 = self.Conv2D_bn(branch5x5, 64, 5, strides=(2, 2))

        # Do convolution with 1x1 filter
        branch3x3dbl = self.Conv2D_bn(x, 64, 1)
        # Do convolution with 3x3 filter
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3)
        # Do convolution with 3x3 filter stride 2x2
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3, strides=(2, 2))

        # Do 3x3 averagepooling with stride 2x2
        branch_pool = AveragePooling2D((3, 3), strides=(2, 2), padding="same")(x)
        # Do convolution with 1x1 filter
        branch_pool = self.Conv2D_bn(branch_pool, 64, 1)
        
        # Concatenate resulting activations along the filters axis
        return concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=-1)


class ResNet(CNN):
    """Model based on ResNet by using shortcut connections"""
    def __init__(self, outputs, input_shape, lr=0.001, decay=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        
        # Shortcut connections allow gradients to propagate more easily through deep networks
        # conv_block has a convolutional layer at the shortcut connection
        # identity_block has no convolutional layer at the shortcut connection
        
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = conv_block(img_input, 3, [32, 32, 128], stage=1, block="a", strides=(1, 1))
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="b")
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="c")

        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = conv_block(x, 3, [64, 64, 256], stage=2, block="a")
        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="b")
        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="c")

        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = conv_block(x, 3, [128, 128, 512], stage=3, block="a")
        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="b")
        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="c")

        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = conv_block(x, 3, [256, 256, 1024], stage=4, block="a")
        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="b")
        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="c")

        # Do GlobalAveragePooling to reduce the 6x6x1024 output to 1x1024
        x = GlobalAveragePooling2D(name="global_avg_pool")(x)
        # Use dense layer to map to number of outputs followed by softmax
        predictions = Dense(outputs, activation="softmax")(x)

        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)
        
        
class DenseNet(CNN):
    """Model based on DenseNet"""
    def __init__(self, outputs, input_shape, lr=0.01, decay=0, dropout=0):
        self.blocks = [3, 6, 12, 8]
        img_input = Input(shape=input_shape)
        x = dense_block(img_input, self.blocks[0], name="conv1")
        x = transition_block(x, 0.5, name="pool1")
        x = dense_block(x, self.blocks[1], name="conv2")
        x = transition_block(x, 0.5, name="pool2")
        x = dense_block(x, self.blocks[2], name="conv3")
        x = transition_block(x, 0.5, name="pool3")
        x = dense_block(x, self.blocks[3], name="conv4")
        
        x = BatchNormalization()(x)
        x = GlobalAveragePooling2D(name="global_avg_pool")(x)
        x = Dropout(dropout)(x)
        predictions = Dense(outputs, activation="softmax")(x)
        
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

### Train models for ensemble

In [19]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 128
EPOCHS = 50

# Create ImageDataGenerator for data augmentation
# Rescale image to take values between 0 and 1
# Allow horizontal flipping, x&y shifting, and zooming
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10)

# Create ImageDataGenerator for validation data
# Rescale image to take values between 0 and 1
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Apply generator to training data
train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True)

# Apply generator to validation data
validation_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)


def train(cnn, epochs=EPOCHS, batch_size=BATCH_SIZE):
    model_name = type(cnn).__name__
    print("Training {}".format(model_name))
    # Create callbacks for checkpointing model progress
    callbacks = [
        TensorBoard(), 
        ModelCheckpoint("data/{}.h5".format(model_name), monitor="val_acc", save_best_only=True, save_weights_only=True),
        EarlyStopping(monitor="val_acc", patience=20)
    ]
    # Train each cnn
    cnn.model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size + 1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size + 1,
        callbacks=callbacks)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [20]:
vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.5, decay=0.0001)
train(vgg)

Training VGG
Epoch 1/50
225/225 [==============================] - 38s 171ms/step - loss: 1.9439 - acc: 0.2562 - val_loss: 3.8721 - val_acc: 0.2449
Epoch 2/50
225/225 [==============================] - 36s 159ms/step - loss: 1.6385 - acc: 0.3992 - val_loss: 13.6105 - val_acc: 0.1323
Epoch 3/50
225/225 [==============================] - 36s 159ms/step - loss: 1.4695 - acc: 0.4641 - val_loss: 1.4582 - val_acc: 0.4152
Epoch 4/50
225/225 [==============================] - 36s 159ms/step - loss: 1.2909 - acc: 0.5115 - val_loss: 1.4263 - val_acc: 0.4656
Epoch 5/50
225/225 [==============================] - 36s 160ms/step - loss: 1.2082 - acc: 0.5425 - val_loss: 1.4442 - val_acc: 0.4748
Epoch 6/50
225/225 [==============================] - 36s 160ms/step - loss: 1.1453 - acc: 0.5701 - val_loss: 1.2251 - val_acc: 0.5483
Epoch 7/50
225/225 [==============================] - 36s 160ms/step - loss: 1.1035 - acc: 0.5853 - val_loss: 1.1476 - val_acc: 0.5614
Epoch 8/50
225/225 [=====================

In [21]:
inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.4, decay=0.001)
train(inception)

Training Inception_FCN
Epoch 1/50
225/225 [==============================] - 49s 219ms/step - loss: 1.7393 - acc: 0.2971 - val_loss: 1.6789 - val_acc: 0.3193


KeyboardInterrupt: 

In [ ]:
resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, decay=0.001)
train(resnet)

In [ ]:
densenet = DenseNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, decay=0.001, dropout=0.2)
train(densenet)

### Check ensemble validation accuracy and produce outputs

In [13]:
# Apply generator to validation data
validation_generator = test_datagen.flow_from_directory(
    "data/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=1,
    class_mode="categorical",
    shuffle=False)

def predict_ensemble(models, img, weights):
    """Average the predictions of all models to get ensemble prediction.
    """
    # Create empty array for model predictions
    predictions = np.zeros((len(models), OUTPUTS))
    # Predict the class of the image for each model
    for i in range(len(models)):
        predictions[i] = models[i].predict(img)
    # Return the weighted average prediction of all models
    return np.argmax(np.average(predictions, axis=0, weights=weights)).squeeze()


# Load 3 trained models
vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
vgg.model.load_weights("data/VGG.h5")
inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
inception.model.load_weights("data/Inception_FCN.h5")
resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, decay=0)
resnet.model.load_weights("data/ResNet.h5")
densenet = DenseNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, decay=0)
densenet.model.load_weights("data/DenseNet.h5")
models = [vgg.model, inception.model, resnet.model, densenet.model]
weights = [0.25, 0.25, 0.25, 0.25]

# Create numpy array for predictions
predictions = np.zeros(validation_generator.n)
# Create numpy array for targets
targets = np.zeros(validation_generator.n)

# Iterate through all validation data
for i, (img, target) in enumerate(validation_generator):
    if i >= validation_generator.n:
        break
    # Get prediction of ensemble
    predictions[i] = predict_ensemble(models, img, weights)
    # Get target
    targets[i] = np.argmax(target, axis=1)
print("Accuracy: {}".format(np.mean(targets == predictions)))

Found 3589 images belonging to 7 classes.
Accuracy: 0.721928113680691
